In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Flatten, MaxPooling1D, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
#from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# df = pd.read_csv("aapl_data.csv")
df = pd.read_csv("sp500_data_v2.csv")
df.index = pd.to_datetime(df["Date"])
df = df.drop("Date",axis=1)

In [ ]:
dfm = df.resample("M").mean()

dfm = dfm[:-1] # As we said, we do not consider the month of end_date

print(dfm.head())
print(dfm.tail())

               Close      High       Low      Open        Volume
Date                                                            
1980-12-31  0.104537  0.104967  0.104537  0.104669  1.034501e+08
1981-01-31  0.108701  0.109294  0.108701  0.109048  2.899947e+07
1981-02-28  0.090683  0.091248  0.090683  0.090932  1.692733e+07
1981-03-31  0.085283  0.085907  0.085283  0.085634  3.185076e+07
1981-04-30  0.093493  0.093984  0.093493  0.093698  2.556800e+07
                 Close        High         Low        Open        Volume
Date                                                                    
2024-12-31  249.049032  250.535969  246.602673  248.058694  4.656743e+07
2025-01-31  234.329359  237.312080  231.583879  234.844792  6.001458e+07
2025-02-28  238.446972  240.519476  235.255780  237.307757  4.538275e+07
2025-03-31  222.702858  225.947619  219.753809  223.035714  5.310664e+07
2025-04-30  201.183811  205.282381  196.250476  199.765714  7.649944e+07


<ipython-input-67-58a790599e5f>:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dfm = df.resample("M").mean()


In [ ]:
# start_date = datetime.datetime(1980, 12, 12) # AAPL
start_date = datetime.datetime(1982, 4, 20) # S&P500
end_date = datetime.datetime(2025, 5, 6)

start_year = start_date.year
start_month = start_date.month
start_day = start_date.day
end_year = end_date.year
end_month = end_date.month
end_day = end_date.day

first_days=[]
# First year
for month in range(start_month, 13):
    first_days.append(df.loc[f"{start_year}-{month:02d}"].index.min())
# Other years
for year in range(start_year+1,end_year):
    for month in range(1,13):
        first_days.append(df.loc[f"{year}-{month:02d}"].index.min())
# Last year
for month in range(1,end_month+1):
    first_days.append(df.loc[f"{end_year}-{month:02d}"].index.min())

In [ ]:
dfm["fd_cm"] = first_days[:-1]
dfm["fd_nm"] = first_days[1:]
dfm["fd_cm_open"] = np.array(df.loc[first_days[:-1],"Open"])
dfm["fd_nm_open"] = np.array(df.loc[first_days[1:],"Open"])
dfm["rapp"] = dfm["fd_nm_open"].divide(dfm["fd_cm_open"])

In [ ]:
print(dfm.head())
print(dfm.tail())

               Close      High       Low      Open        Volume      fd_cm  \
Date                                                                          
1980-12-31  0.104537  0.104967  0.104537  0.104669  1.034501e+08 1980-12-12   
1981-01-31  0.108701  0.109294  0.108701  0.109048  2.899947e+07 1981-01-02   
1981-02-28  0.090683  0.091248  0.090683  0.090932  1.692733e+07 1981-02-02   
1981-03-31  0.085283  0.085907  0.085283  0.085634  3.185076e+07 1981-03-02   
1981-04-30  0.093493  0.093984  0.093493  0.093698  2.556800e+07 1981-04-01   

                fd_nm  fd_cm_open  fd_nm_open      rapp  
Date                                                     
1980-12-31 1981-01-02    0.098726    0.118471  1.200004  
1981-01-31 1981-02-02    0.118471    0.091858  0.775364  
1981-02-28 1981-03-02    0.091858    0.091429  0.995327  
1981-03-31 1981-04-01    0.091429    0.083703  0.915490  
1981-04-30 1981-05-01    0.083703    0.097438  1.164101  
                 Close        High      

In [ ]:
dfm["mv_avg_12"]= dfm["Open"].rolling(window=12).mean().shift(1)
dfm["mv_avg_24"]= dfm["Open"].rolling(window=24).mean().shift(1)

In [ ]:
print(dfm.loc["1984-03","mv_avg_12"])
print(dfm.loc["1983-03":"1984-02","Open"])
print(dfm.loc["1983-03":"1984-02","Open"].mean())

Date
1984-03-31    0.121552
Freq: ME, Name: mv_avg_12, dtype: float64
Date
1983-03-31    0.149563
1983-04-30    0.155858
1983-05-31    0.186701
1983-06-30    0.192691
1983-07-31    0.151952
1983-08-31    0.114310
1983-09-30    0.104879
1983-10-31    0.073135
1983-11-30    0.071071
1983-12-31    0.078020
1984-01-31    0.094107
1984-02-29    0.086342
Freq: ME, Name: Open, dtype: float64
0.12155247040896244


In [ ]:
dfm=dfm.iloc[24:,:] # WARNING: DO IT JUST ONE TIME!
print(dfm.index)

DatetimeIndex(['1982-12-31', '1983-01-31', '1983-02-28', '1983-03-31',
               '1983-04-30', '1983-05-31', '1983-06-30', '1983-07-31',
               '1983-08-31', '1983-09-30',
               ...
               '2024-07-31', '2024-08-31', '2024-09-30', '2024-10-31',
               '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28',
               '2025-03-31', '2025-04-30'],
              dtype='datetime64[ns]', name='Date', length=509, freq='ME')


In [ ]:
mtest=72
train=dfm.iloc[:-mtest,:]
test=dfm.iloc[-mtest:,:]

In [ ]:
# This function returns the total percentage gross yield and the annual percentage gross yield

def yield_gross(df,v):
    prod=(v*df["rapp"]+1-v).prod()
    n_years=len(v)/12
    return (prod-1)*100,((prod**(1/n_years))-1)*100

In [ ]:
tax_cg = 0.26
comm_bk = 0.001

In [ ]:
# This function will be used in the function yield_net

# Given any vector v of ones and zeros, this function gives the corresponding vectors of "islands" of ones of v
# and their number.
# For example, given v = [0,1,1,0,1,0,1], expand_islands2D gives
# out2D = [[0,1,1,0,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,1]] and N=3

def expand_islands2D(v):

    # Get start, stop of 1s islands
    v1 = np.r_[0,v,0]
    idx = np.flatnonzero(v1[:-1] != v1[1:])
    s0,s1 = idx[::2],idx[1::2]
    if len(s0)==0:
        return np.zeros(len(v)),0

    # Initialize 1D id array  of size same as expected o/p and has
    # starts and stops assigned as 1s and -1s, so that a final cumsum
    # gives us the desired o/p
    N,M = len(s0),len(v)
    out = np.zeros(N*M,dtype=int)

    # Setup starts with 1s
    r = np.arange(N)*M
    out[s0+r] = 1


    # Setup stops with -1s
    if s1[-1] == M:
        out[s1[:-1]+r[:-1]] = -1
    else:
        out[s1+r] -= 1

    # Final cumsum on ID array
    out2D = out.cumsum().reshape(N,-1)
    return out2D,N

In [ ]:
# This function returns the total percentage net yield and the annual percentage net yield

def yield_net(df,v):
    n_years=len(v)/12

    w, n = expand_islands2D(v)
    A = (w*np.array(df["rapp"])+(1-w)).prod(axis=1)  # A is the product of each island of ones of 1 for df["rapp"]
    A1p = np.maximum(0,np.sign(A-1)) # vector of ones where the corresponding element if  A  is > 1, other are 0
    Ap = A*A1p # vector of elements of A > 1, other are 0
    Am = A-Ap # vector of elements of A <= 1, other are 0
    An = Am+(Ap-A1p)*(1-tax_cg)+A1p
    prod = An.prod()*((1-comm_bk)**(2*n))

    return (prod-1)*100,((prod**(1/n_years))-1)*100

In [ ]:
def create_window(data, window_size = 1):
    data_s = data.copy()
    for i in range(window_size):
        data = pd.concat([data, data_s.shift(-(i + 1))], axis = 1)

    data.dropna(axis=0, inplace=True)
    return(data)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
dg = pd.DataFrame(scaler.fit_transform(dfm[["High","Low","Open","Close","Volume","fd_cm_open",\
                                          "mv_avg_12","mv_avg_24","fd_nm_open"]].values))
dg0 = dg[[0,1,2,3,4,5,6,7]]


window = 4
dfw = create_window(dg0, window)

X_dfw = np.reshape(dfw.values, (dfw.shape[0], window+1,8))
print(X_dfw.shape)
print(dfw.iloc[:4,:])
print(X_dfw[0,:,:])

y_dfw = np.array(dg[8][window:])

(505, 5, 8)
          0         1         2         3         4         5         6  \
0  0.000225  0.000213  0.000219  0.000219  0.047391  0.000233  0.000000   
1  0.000261  0.000243  0.000248  0.000256  0.100859  0.000205  0.000014   
2  0.000414  0.000398  0.000406  0.000409  0.071276  0.000357  0.000031   
3  0.000396  0.000384  0.000392  0.000391  0.042414  0.000423  0.000065   

              7         0         1  ...         6         7         0  \
0  0.000000e+00  0.000261  0.000243  ...  0.000065  0.000014  0.000427   
1  3.893744e-07  0.000414  0.000398  ...  0.000100  0.000028  0.000549   
2  1.394359e-06  0.000396  0.000384  ...  0.000138  0.000041  0.000570   
3  1.438766e-05  0.000427  0.000413  ...  0.000190  0.000059  0.000406   

          1         2         3         4         5         6         7  
0  0.000413  0.000418  0.000423  0.071832  0.000376  0.000100  0.000028  
1  0.000535  0.000542  0.000546  0.033391  0.000490  0.000138  0.000041  
2  0.000552  0.0005

In [ ]:
X_trainw = X_dfw[:-mtest-1, :, :]
X_testw = X_dfw[-mtest-1:, :, :]
y_trainw = y_dfw[:-mtest-1]
y_testw = y_dfw[-mtest-1:]

In [ ]:
def model_lstm(window,features):

    model = Sequential()
    model.add(LSTM(300, input_shape = (window, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(200, input_shape=(window, features), return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='relu'))
    model.compile(loss='mse', optimizer='adam')

    return model

In [ ]:
model = model_lstm(window+1,8)
# history = model.fit(X_trainw, y_trainw, epochs=100, batch_size=1, validation_data=(X_testw, y_testw), \
                  # verbose=1, callbacks=[], shuffle=False) # AAPL
history = model.fit(X_trainw, y_trainw, epochs=100, batch_size=6, validation_data=(X_testw, y_testw), \
                  verbose=1, callbacks=[], shuffle=False) # S&P 500 profitable

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
y_pr = model.predict(X_trainw)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(y_trainw, label="actual")
plt.plot(y_pr, label="prediction")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Actual open price and pedicted one on train set",fontsize=25)
plt.show()

In [ ]:
y_pred=model.predict(X_testw)

In [ ]:
v=np.diff(y_pred.reshape(y_pred.shape[0]),1)
v_lstm=np.maximum(np.sign(v),0)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(y_testw, label="actual")
plt.plot(y_pred, label="prediction")
plt.plot(v_lstm,label="In and out")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Actual open price, predicted one and vector v_lstm",fontsize=25)
plt.show()

In [ ]:
v_bh=np.ones(test.shape[0])
v_ma=test["fd_cm_open"]>test["mv_avg_12"]

In [ ]:
def gross_portfolio(df,w):
    portfolio=[ (w*df["rapp"]+(1-w))[:i].prod() for i in range(len(w))]
    return portfolio

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(gross_portfolio(test,v_bh),label="Portfolio Buy and Hold")
plt.plot(gross_portfolio(test,v_ma),label="Portfolio Moving Average")
plt.plot(gross_portfolio(test,v_lstm),label="Portfolio LSTM")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Gross portfolios of three methods", fontsize=25)
plt.show()

In [ ]:
print("Test period of {:.2f} years, from {} to {} \n".format(len(v_bh)/12,str(test.loc[test.index[0],"fd_cm"])[:10],\
      str(test.loc[test.index[-1],"fd_nm"])[:10]))

results0=pd.DataFrame({})
results1=pd.DataFrame({})
results2=pd.DataFrame({})
results3=pd.DataFrame({})

results0["Method"]=["Buy and hold","Moving average","LSTM"]
results1["Method"]=["Buy and hold","Moving average","LSTM"]
results2["Method"]=["Buy and hold","Moving average","LSTM"]
results3["Method"]=["Buy and hold","Moving average","LSTM"]

vs=[v_bh,v_ma,v_lstm]
results0["Total gross yield"]=[str(round(yield_gross(test,vi)[0],2))+" %" for vi in vs]
results1["Annual gross yield"]=[str(round(yield_gross(test,vi)[1],2))+" %" for vi in vs]
results2["Total net yield"]=[str(round(yield_net(test,vi)[0],2))+" %" for vi in vs]
results3["Annual net yield"]=[str(round(yield_net(test,vi)[1],2))+" %" for vi in vs]

print(results0)
print("\n")
print(results1)
print("\n")
print(results2)
print("\n")
print(results3)